## Previsão de Partidas

Modelagem não tem a pretenção de fazer uma representação do real, mas entregar um equacionamento útil. Por isso, tentaremos quantificar os placares que podem ocorrer.

A variância no futebol é tão grande que por isso o torna interessante, essa variância coloca nos detalhes as questões das partidas. Se pudessemos repetir as partidas diversas vezes, provavelmente teriamos diversas partidas com valores diferentes.

Quando trabalhando com futebol, estamos quantificando todas as incertezas que podem acontecer.

### Problema 1: Como Modelar a Distribuição de Gols de um Time em uma Partida

##### Distribuição Poisson:

Ajuda quando estamos tentando quantificar a probabilidade de contagens. É uma boa opção para dados de futebol, para contagens de gols em partidas, isso foi verificado por diversos pesquisadores.

<img src="https://1.bp.blogspot.com/-1zgtrAq0M8s/V9_8r65F20I/AAAAAAAAAJM/Xfp41CV21donWbfnD4qZX8mnBCvHf3T3wCLcB/s1600/qqq.png">

Onde:  
k - número de ocorrências;  
e - número de Euler.  

Com isso, podemos calcular a probabilidade de cada placar multiplicando a probabilidade de determinado time fazer aquela quantidade de gols pela probabilidade do outro time fazer tal quantidade de gols.

<img src="https://i.ibb.co/CBhWp1P/smd.png">

A Probabilidade de Empate na partida é a soma das probabilidades de empate em cada resultado.  
A Probabilidade de Vitória do Liverpool é a soma das probabilidades das combinações de placares em que ele vence.  
A Probabilidade de Vitória do Brighton é a soma das probabilidades de todas as combinações em que ele vence.

Dessa forma, a distribuição de gols das seleções 1 e 2, respectivamente, em um jogo
    S¹ ~ Poisson(m1)
    s² ~ Poisson(m2)
Onde:  
m - média de gols.

Em que S¹ e S² são independentes.

### Problema 2: Como Descobri a Média de Gols de Cada Time em uma Partida

Partindo da média histórica de gols em copa do mundo e considerando alguns fatores, podemos considerar a média de gols esperados na copa do mundo em 2,75.

<img src="https://uploaddeimagens.com.br/images/004/172/056/full/Captura_da_Web_22-11-2022_11926_ge.globo.com.jpeg?1669090784">

    m = m¹ + m²
    m = 2,75
    
Agora basta descobrir para cada equipe, quanto que fica para cada um.

Para isso, usaremos o fator de força para cada equipe

    m¹ = m . f¹ / (f¹ + f²)
    m² = m - m²
    
Onde:  
f¹ - força da Seleção 1  
f² - força da Seleção 2

A forma de cálculo da força de cada seleção será utilizar o Ranking Fifa. Transformaremos os valores do ranking para um intervalo (a, 1), com 1 sendo a seleção mais forte. 

In [1]:
import pandas as pd
import numpy as np
import random
from scipy.stats import poisson

In [2]:
selecoes = pd.read_excel('DadosCopaDoMundoQatar2022.xlsx', sheet_name='selecoes', index_col=0)
jogos = pd.read_excel('DadosCopaDoMundoQatar2022.xlsx', sheet_name='jogos')

In [3]:
selecoes.head()

,Grupo,GrupoNumero,NomeEmIngles,PosiçãoRankingFIFA,ValorDeMercado,PontosRankingFIFA,Confederação,Copas,JogadorDestaque,FotoJogadorDestaque,LinkBandeiraPequena,LinkBandeiraGrande
Seleção,,,,,,,,,,,,
Catar,A,A1,Qatar,48,18.33,1442,AFC,0,Almoez Ali,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Equador,A,A2,Ecuador,44,125.80,1464,UEFA,0,Pervis Estupiñán,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Senegal,A,A3,Senegal,18,286.50,1585,CAF,0,Sadio Mané,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Holanda,A,A4,Netherlands,8,455.75,1679,UEFA,0,Virgil Van Dijk,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Inglaterra,B,B1,England,5,1360.00,1737,UEFA,1,Harry Kane,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...


In [4]:
jogos.head()

,data,hora,grupo,seleção1,seleção2,estádio
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR


In [5]:
# Transformação Linear de Escala Numérica
fifa = selecoes['PontosRankingFIFA']
a, b = min(fifa), max(fifa)
fa, fb = 0.15, 1

b1 = (fb - fa)/(b-a)
b0 = fb - b*b1
forca = b0 + b1*fifa

In [6]:
forca

Seleção
Catar             0.243596
Equador           0.285618
Senegal           0.516742
Holanda           0.696292
Inglaterra        0.807079
Irã               0.467079
Estados Unidos    0.612247
País de Gales     0.511011
Argentina         0.872022
Arábia Saudita    0.232135
México            0.640899
Polônia           0.442247
França            0.860562
Dinamarca         0.669551
Tunísia           0.369663
Austrália         0.323820
Espanha           0.768876
Alemanha          0.658090
Japão             0.459438
Costa Rica        0.354382
Bélgica           0.969438
Canadá            0.304719
Marrocos          0.465169
Croácia           0.606517
Brasil            1.000000
Sérvia            0.449888
Suíça             0.585506
Camarões          0.325730
Portugal          0.696292
Gana              0.150000
Uruguai           0.623708
Coreia do Sul     0.404045
Name: PontosRankingFIFA, dtype: float64

In [7]:
# Encontrando M1 e M2

def MediasPoisson(selecao1, selecao2):
    forca1 = forca[selecao1]
    forca2 = forca[selecao2]
    mgols = 2.75
    
    l1 = mgols*forca1/(forca1+forca2)
    l2 = mgols - l1
    return l1, l2

def Resultado(gols1, gols2):
    if gols1 > gols2:
        resultado = 'V'
    elif gols1 == gols2:
        resultado = 'E'
    else:
        resultado = 'D'
    return resultado

def Pontos(gols1, gols2):
    rst = Resultado(gols1, gols2)
    if rst == 'V':
        pontos1, pontos2 = 3, 0
    if rst == 'E':
        pontos1, pontos2 = 1, 1
    if rst == 'D':
        pontos1, pontos2 = 0, 3
    return [pontos1, pontos2, rst]

In [8]:
Pontos(0,2)

[0, 3, 'D']

In [9]:
# É por meio dessa diferença de forças que os placares se diferenciam para cada partida.
print('Brasil x Argentina')
print(MediasPoisson('Brasil', 'Argentina'))

print('Catar x Equador')
print(MediasPoisson('Catar', 'Equador'))

Brasil x Argentina
(1.4689994598163376, 1.2810005401836624)
Catar x Equador
(1.2658174097664538, 1.4841825902335462)


In [10]:
def Jogo(selecao1, selecao2):
    l1, l2 = MediasPoisson(selecao1, selecao2)
    
    gols1 = int(np.random.poisson(lam = l1, size=1))
    gols2 = int(np.random.poisson(lam = l2, size=1))
    
    saldo1 = gols1 - gols2
    saldo2 = gols2 - gols1
    
    pontos1, pontos2, resultado = Pontos(gols1, gols2)
    
    placar = '{}x{}'.format(gols1, gols2)
    
    return [gols1, gols2, saldo1, saldo2, pontos1, pontos2, resultado, placar]

In [11]:
def Distribuicao(media):
    probs = []
    
    for i in range(7):
        probs.append(poisson.pmf(i, media))
    probs.append(1 - sum(probs))
    return pd.Series(probs, index=['0', '1', '2', '3', '4', '5', '6', '7+'])

def ProbabilidadesPartidas(selecao1, selecao2):
    l1, l2 = MediasPoisson(selecao1, selecao2)
    d1, d2 = Distribuicao(l1), Distribuicao(l2)
    matriz = np.outer(d1, d2)
    vitoria = np.tril(matriz).sum() - np.trace(matriz)  #Soma o triâgulo inferior
    derrota = np.triu(matriz).sum() - np.trace(matriz)  #Soma o triângulo superior
    empate = 1 - (vitoria + derrota)
    
    probs = np.around([vitoria, empate, derrota], 3)
    probsp = [f'{100*i:.1f}%' for i in probs]
    
    nomes = ['0', '1', '2', '3', '4', '5', '6', '7+']
    matriz = pd.DataFrame(matriz, columns=nomes, index=nomes)
    matriz.index = pd.MultiIndex.from_product([[selecao1], matriz.index])
    matriz.columns = pd.MultiIndex.from_product([[selecao2], matriz.columns])
    
    output = {
        'seleção1': selecao1,
        'seleção2': selecao2,
        'f1': forca[selecao1],
        'f2': forca[selecao2],
        'media1': l1,
        'media2': l2,
        'probabilidades': probsp,
        'matriz': matriz
    }
    return output

In [12]:
ProbabilidadesPartidas('Brasil', 'França')

{'seleção1': 'Brasil',
 'seleção2': 'França',
 'f1': 1.0,
 'f2': 0.8605617977528088,
 'media1': 1.478048191315901,
 'media2': 1.271951808684099,
 'probabilidades': ['42.0%', '25.4%', '32.6%'],
 'matriz':              França                                                    \
                   0         1         2         3         4         5   
 Brasil 0   0.063928  0.081313  0.051713  0.021926  0.006972  0.001774   
        1   0.094488  0.120185  0.076435  0.032407  0.010305  0.002622   
        2   0.069829  0.088819  0.056487  0.023950  0.007616  0.001937   
        3   0.034404  0.043760  0.027830  0.011800  0.003752  0.000955   
        4   0.012713  0.016170  0.010284  0.004360  0.001386  0.000353   
        5   0.003758  0.004780  0.003040  0.001289  0.000410  0.000104   
        6   0.000926  0.001177  0.000749  0.000318  0.000101  0.000026   
        7+  0.000239  0.000303  0.000193  0.000082  0.000026  0.000007   
 
                                    
                  

In [13]:
jogos['Vitória'] = None
jogos['Empate'] = None
jogos['Derrota'] = None

for i in range(jogos.shape[0]):
    selecao1 = jogos['seleção1'][i]
    selecao2 = jogos['seleção2'][i]
    
    v, e, d = ProbabilidadesPartidas(selecao1, selecao2)['probabilidades']
    jogos.at[i, 'Vitória'] = v
    jogos.at[i, 'Derrota'] = d
    jogos.at[i, 'Empate'] = e

In [14]:
jogos

,data,hora,grupo,seleção1,seleção2,estádio,Vitória,Empate,Derrota
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA,28.3%,24.9%,46.8%
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR,32.3%,25.4%,42.3%
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA,17.2%,22.1%,60.6%
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA,64.4%,21.0%,14.6%
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR,68.4%,19.6%,12.0%
5,2022-11-29,12:00:00,A,Equador,Senegal,INTER. KHALIFA,20.7%,23.3%,56.0%
6,2022-11-21,10:00:00,B,Inglaterra,Irã,INTER. KHALIFA,54.6%,23.6%,21.8%
7,2022-11-21,16:00:00,B,Estados Unidos,País de Gales,AL RAYYAN,43.0%,25.3%,31.7%
8,2022-11-25,07:00:00,B,País de Gales,Irã,AL RAYYAN,40.1%,25.5%,34.4%
9,2022-11-25,16:00:00,B,Inglaterra,Estados Unidos,AL KHOR,46.1%,25.0%,28.9%


In [15]:
jogos.to_excel('estimativasJogosCopa.xlsx')

## Previsão da Copa do Mundo

### Primeira Fase - Fase de Grupos

In [16]:
def JogosGrupo(dados, grupo):

    times = list(dados[dados['Grupo'] == grupo].index)
    time1, time2, time3, time4 = times

    jogo1 = Jogo(time1, time2)
    jogo2 = Jogo(time3, time4)
    jogo3 = Jogo(time1, time3)
    jogo4 = Jogo(time2, time4)
    jogo5 = Jogo(time1, time4)
    jogo6 = Jogo(time2, time3)

    pt1, pt2, pt3, pt4 = 0, 0, 0, 0
    gp1, gp2, gp3, gp4 = 0, 0, 0, 0
    sg1, sg2, sg3, sg4 = 0, 0, 0, 0

    gp1, gp2, sg1, sg2, pt1, pt2 = gp1 + jogo1[0], gp2 + jogo1[1], sg1 + jogo1[2], sg2 + jogo1[3], pt1 + jogo1[4], pt2 + jogo1[5]
    gp3, gp4, sg3, sg4, pt3, pt4 = gp3 + jogo2[0], gp4 + jogo2[1], sg3 + jogo2[2], sg4 + jogo2[3], pt3 + jogo2[4], pt4 + jogo2[5]
    gp1, gp3, sg1, sg3, pt1, pt3 = gp1 + jogo3[0], gp3 + jogo3[1], sg1 + jogo3[2], sg3 + jogo3[3], pt1 + jogo3[4], pt3 + jogo3[5]
    gp2, gp4, sg2, sg4, pt2, pt4 = gp2 + jogo4[0], gp4 + jogo4[1], sg2 + jogo4[2], sg4 + jogo4[3], pt2 + jogo4[4], pt4 + jogo4[5]
    gp1, gp4, sg1, sg4, pt1, pt4 = gp1 + jogo5[0], gp4 + jogo5[1], sg1 + jogo5[2], sg4 + jogo5[3], pt1 + jogo5[4], pt4 + jogo5[5]
    gp2, gp3, sg2, sg3, pt2, pt3 = gp2 + jogo6[0], gp3 + jogo6[1], sg2 + jogo6[2], sg3 + jogo6[3], pt2 + jogo6[4], pt3 + jogo6[5]

    tab = pd.DataFrame([[pt1, pt2, pt3, pt4], [sg1, sg2, sg3, sg4], [gp1, gp2, gp3, gp3]], 
                       columns=[time1, time2, time3, time4],
                       index = ['Pontos', 'Saldo de Gols', 'Gols Pró']).T


    tab = tab.sort_values(by = ['Pontos', 'Saldo de Gols', 'Gols Pró'], ascending=False)
    tab['Posição'] = [1, 2, 3, 4]

    partidas = [
        time1 + ' x ' + time2,
        time3 + ' x ' + time4,
        time1 + ' x ' + time3,
        time2 + ' x ' + time4,
        time1 + ' x ' + time4,
        time2 + ' x ' + time3,
    ]

    resultados = [jogo1[6], jogo2[6], jogo3[6], jogo4[6], jogo5[6], jogo6[6]]
    placares = [jogo1[-1], jogo2[-1], jogo3[-1], jogo4[-1], jogo5[-1], jogo6[-1]]

    jogos = pd.DataFrame([partidas, placares, resultados]).transpose()
    jogos.columns = ['Partidas', 'Placar', 'Resultado']

    return [tab, jogos]

### Segunda Fase - Fase Eliminatória

In [17]:
def JogoMataMata(selecao1, selecao2):
    
    jogo = Jogo(selecao1, selecao2)
    resultado = jogo[6]
    
    if resultado == 'V':
        return selecao1
    elif resultado == 'D':
        return selecao2
    else:
        return random.sample([selecao1, selecao2], 1)[0]

In [18]:
def SimulaCopa(selecoes):
    cols = ['1st', '2sd', '3rd', '4th', 'Oitavas', 'Quartas', 'Semis', 'Final', 'Campeão']

    n = selecoes.shape[0]
    m = len(cols)
    aux = np.array(np.zeros(n*m).reshape(n, m))
    info = pd.DataFrame(aux, columns=cols, index=selecoes.index)
    info = info.astype(int)

    top16 = []
    for i in list('ABCDEFGH'):
        a = JogosGrupo(selecoes, grupo = i)[0]
        a.index[:2].tolist()

        top16 += a.index[:2].tolist()
        anomes = a.index.tolist()
        info.at[anomes[0], '1st'] = 1
        info.at[anomes[1], '2sd'] = 1
        info.at[anomes[2], '3rd'] = 1
        info.at[anomes[3], '4th'] = 1

    qf1 = JogoMataMata(top16[0], top16[3])    
    qf2 = JogoMataMata(top16[2], top16[1])    
    qf3 = JogoMataMata(top16[4], top16[7])    
    qf4 = JogoMataMata(top16[6], top16[5])    
    qf5 = JogoMataMata(top16[8], top16[11])    
    qf6 = JogoMataMata(top16[10], top16[9])
    qf7 = JogoMataMata(top16[12], top16[15])
    qf8 = JogoMataMata(top16[14], top16[13])

    top8 = [qf1, qf2, qf3, qf4, qf5, qf6, qf7, qf8]

    sf1 = JogoMataMata(top8[0], top8[3])
    sf2 = JogoMataMata(top8[2], top8[1])
    sf3 = JogoMataMata(top8[4], top8[7])
    sf4 = JogoMataMata(top8[6], top8[5])

    top4 = [sf1, sf2, sf3, sf4]

    f1 = JogoMataMata(sf1, sf3)
    f2 = JogoMataMata(sf2, sf4)

    top2 = [f1, f2]

    top1 = JogoMataMata(f1, f2)


    info.loc[top16, 'Oitavas'] = 1
    info.loc[top8, 'Quartas'] = 1
    info.loc[top4, 'Semis'] = 1
    info.loc[top2, 'Final'] = 1
    info.loc[top1, 'Campeão'] = 1

    return info

In [19]:
def SimulacaoTotal(dados, S = 1000):
    
    print('IA: "Iniciando Simulação..."')
    info = SimulaCopa(selecoes)
    for i in range(S - 1):
        info += SimulaCopa(selecoes)
        if (i+2)%(S/10) == 0:
            print('IA: Simulação de Copa do Mundo: {:.0f}% completas'.format(100*((i+2)/S)))
    print('IA: "Simulação Finalizada!"')
    
    return info.sort_values(by = 'Campeão', ascending=False)/S

In [20]:
S = 10000
sim = SimulacaoTotal(selecoes, S)
sim.to_excel('outputSimulacoesCopadoMundo.xlsx')
sim

IA: "Iniciando Simulação..."
IA: Simulação de Copa do Mundo: 10% completas
IA: Simulação de Copa do Mundo: 20% completas
IA: Simulação de Copa do Mundo: 30% completas
IA: Simulação de Copa do Mundo: 40% completas
IA: Simulação de Copa do Mundo: 50% completas
IA: Simulação de Copa do Mundo: 60% completas
IA: Simulação de Copa do Mundo: 70% completas
IA: Simulação de Copa do Mundo: 80% completas
IA: Simulação de Copa do Mundo: 90% completas
IA: Simulação de Copa do Mundo: 100% completas
IA: "Simulação Finalizada!"


,1st,2sd,3rd,4th,Oitavas,Quartas,Semis,Final,Campeão
Seleção,,,,,,,,,
Brasil,0.5920,0.2554,0.1094,0.0432,0.8474,0.5690,0.3525,0.2144,0.1349
Bélgica,0.5781,0.2606,0.1167,0.0446,0.8387,0.5425,0.3232,0.1945,0.1126
Argentina,0.5294,0.2844,0.1438,0.0424,0.8138,0.4902,0.3000,0.1711,0.0986
França,0.5217,0.2940,0.1284,0.0559,0.8157,0.4835,0.2947,0.1693,0.0947
Inglaterra,0.4456,0.2743,0.1741,0.1060,0.7199,0.4667,0.2584,0.1397,0.0736
Espanha,0.4510,0.2891,0.1685,0.0914,0.7401,0.4033,0.2232,0.1194,0.0618
Holanda,0.5270,0.2953,0.1317,0.0460,0.8223,0.4409,0.2203,0.1097,0.0506
Portugal,0.4595,0.3175,0.1850,0.0380,0.7770,0.4060,0.2094,0.1065,0.0505
Dinamarca,0.3343,0.3588,0.2068,0.1001,0.6931,0.3532,0.1863,0.0905,0.0433


In [21]:
etapas = pd.DataFrame()
etapas['Cair na 1º Fase'] = 1 - sim['Oitavas']
etapas['Cair nas Oitavas'] = sim['Oitavas'] - sim['Quartas']
etapas['Cair nas Quartas'] = sim['Quartas'] - sim['Semis']
etapas['Cair nas Semis'] = sim['Semis'] - sim['Final']
etapas['Cair nas Final'] = sim['Final'] - sim['Campeão']
etapas['Ganhar a Final'] = sim['Campeão']
etapas.to_excel('outputProbabilidadesPorEtapa.xlsx')
etapas

,Cair na 1º Fase,Cair nas Oitavas,Cair nas Quartas,Cair nas Semis,Cair nas Final,Ganhar a Final
Seleção,,,,,,
Brasil,0.1526,0.2784,0.2165,0.1381,0.0795,0.1349
Bélgica,0.1613,0.2962,0.2193,0.1287,0.0819,0.1126
Argentina,0.1862,0.3236,0.1902,0.1289,0.0725,0.0986
França,0.1843,0.3322,0.1888,0.1254,0.0746,0.0947
Inglaterra,0.2801,0.2532,0.2083,0.1187,0.0661,0.0736
Espanha,0.2599,0.3368,0.1801,0.1038,0.0576,0.0618
Holanda,0.1777,0.3814,0.2206,0.1106,0.0591,0.0506
Portugal,0.2230,0.3710,0.1966,0.1029,0.0560,0.0505
Dinamarca,0.3069,0.3399,0.1669,0.0958,0.0472,0.0433


In [22]:
avanco = pd.DataFrame()
avanco['Avançar na 1ª Fase'] = sim['Oitavas']
avanco['Avançar nas Oitavas'] = sim['Quartas']/sim['Oitavas']
avanco['Avançar nas Quartas'] = sim['Semis']/sim['Quartas']
avanco['Avançar nas Semis'] = sim['Final']/sim['Semis']
avanco['Avançar na Final'] = sim['Campeão']/sim['Final']
avanco.to_excel('outputAvancoPorEtapa.xlsx')
avanco

,Avançar na 1ª Fase,Avançar nas Oitavas,Avançar nas Quartas,Avançar nas Semis,Avançar na Final
Seleção,,,,,
Brasil,0.8474,0.671466,0.619508,0.608227,0.629198
Bélgica,0.8387,0.646834,0.595760,0.601795,0.578920
Argentina,0.8138,0.602359,0.611995,0.570333,0.576271
França,0.8157,0.592742,0.609514,0.574483,0.559362
Inglaterra,0.7199,0.648284,0.553675,0.540635,0.526843
Espanha,0.7401,0.544926,0.553434,0.534946,0.517588
Holanda,0.8223,0.536179,0.499660,0.497957,0.461258
Portugal,0.7770,0.522523,0.515764,0.508596,0.474178
Dinamarca,0.6931,0.509595,0.527463,0.485776,0.478453
